In [1]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
model = BertModel.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [4]:
named_params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(named_params)))

print('===== Embedding Layer =====\n')
for p in named_params[0:5]:
  print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n===== First Encoder =====\n')
for p in named_params[5:21]:
  print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n===== Output Layer =====\n')
for p in named_params[-2:]:
  print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 199 different named parameters.

===== Embedding Layer =====

embeddings.word_embeddings.weight                       (30522, 768)
embeddings.position_embeddings.weight                     (512, 768)
embeddings.token_type_embeddings.weight                     (2, 768)
embeddings.LayerNorm.weight                                   (768,)
embeddings.LayerNorm.bias                                     (768,)

===== First Encoder =====

encoder.layer.0.attention.self.query.weight               (768, 768)
encoder.layer.0.attention.self.query.bias                     (768,)
encoder.layer.0.attention.self.key.weight                 (768, 768)
encoder.layer.0.attention.self.key.bias                       (768,)
encoder.layer.0.attention.self.value.weight               (768, 768)
encoder.layer.0.attention.self.value.bias                     (768,)
encoder.layer.0.attention.output.dense.weight             (768, 768)
encoder.layer.0.attention.output.dense.bias                   (

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [19]:
tokenizer.encode('Sinan loves a beautiful day')

[101, 8254, 2319, 7459, 1037, 3376, 2154, 102]

In [8]:
response = model(torch.tensor([tokenizer.encode('Sinan loves a beautiful day')]))

In [9]:
response.last_hidden_state

tensor([[[-0.2327,  0.1515, -0.0448,  ..., -0.5192,  0.4195,  0.2948],
         [ 0.3051, -0.6614,  0.2500,  ..., -0.9809,  0.2551,  0.2400],
         [-0.3610, -0.8759,  0.4542,  ..., -1.1120,  0.1791,  0.0664],
         ...,
         [ 0.0689, -0.0364,  0.4940,  ..., -0.6558,  0.2227, -0.3868],
         [-0.2657, -0.4257,  0.0056,  ...,  0.1352,  0.3596, -0.4585],
         [ 0.6100,  0.0263, -0.2532,  ..., -0.0680, -0.3901, -0.3541]]],
       grad_fn=<NativeLayerNormBackward0>)

In [10]:
response.pooler_output.shape

torch.Size([1, 768])

In [11]:
model.pooler

BertPooler(
  (dense): Linear(in_features=768, out_features=768, bias=True)
  (activation): Tanh()
)

In [14]:
CLS_embedding = response.last_hidden_state[:,0,:].unsqueeze(0)

CLS_embedding.shape

torch.Size([1, 1, 768])

In [15]:
model.pooler(CLS_embedding).shape

torch.Size([1, 768])

In [16]:
(model.pooler(CLS_embedding) == response.pooler_output).all()

tensor(True)

In [18]:
total_params = 0
for param in model.parameters():
  if len(param.shape) == 2:
    total_params += param.shape[0] * param.shape[1]

print(f'Total number of parameters: {total_params}')

Total number of parameters: 109360128
